Hi all, I have tried to use decision tree classification to predict if someone should be given loan or not. I have also used XGboost feature importance, which can be seen further below. The whole purpose of this was to check that if data preprocessing steps make any improvement in our models or not. 

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import seaborn as sns
from xgboost import XGBClassifier
from xgboost import plot_importance
np.random.seed(0)

In [ ]:
file=pd.read_csv("../input/loan-data-set/loan_data_set.csv")
data=file
data.isnull().sum()

As we can see there are some features that have missing values, so I will impute those missing values with the most frequent of the values in their respective column.

In [ ]:
impute_missing=SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
impute_missing.fit(data)
data=impute_missing.transform(data)
data=pd.DataFrame(data=data,columns=file.columns)
data.isnull().sum()

In [ ]:
encoding=OrdinalEncoder()
X=data
X=encoding.fit_transform(X)
X=pd.DataFrame(data=X,columns=file.columns)
Y=X["Loan_Status"]
X=X.loc[:,X.columns!="Loan_Status"]
copy=X

In [ ]:
model=DecisionTreeClassifier()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=0)
model.fit(X_train,Y_train)
plt.figure(figsize=(20,20))
tree.plot_tree(model.fit(X_train,Y_train))
print("Decision tree accuracy:::::",model.score(X_test,Y_test)*100,"%")

Here i have done only 2 preprocessing steps,** missing value imputation** and **categorical to numnerical encoding**. Still the decision tree is giving a good accuracy. Now we will try to do some more preprocessing and we shall check if there is any improvement or not.

**Standardizing the data**

In [ ]:
Scaler=StandardScaler()
standard_data=Scaler.fit_transform(X)
X=pd.DataFrame(data=standard_data,columns=copy.columns)
X

In [ ]:
model=DecisionTreeClassifier()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=0)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test)*100,"%")

* As we can see that there no increase in the accuracy after standardizing the data, rather there is a decrease!

**Checking for outliers**

In [ ]:
plt.figure(figsize=(17,17))
sns.boxplot(data=pd.DataFrame(X))
plt.show()

No significant outliers can be seen! We wouldn't deal with the outliers for now as they are very small in number

**Feature selection**

In [ ]:
model1=XGBClassifier()
model1.fit(X,Y)
plot_importance(model1)
plt.show()

From the feature importance plot we can see that there are some features which are not that important than others, so we would remove some of the features while training our model.

In [ ]:
sort=model1.get_booster().get_score()
print(sort)
selected_features=["Dependents","Married","Loan_Amount_Term","Credit_History","Property_Area","CoapplicantIncome","LoanAmount","ApplicantIncome"]
## removed --Gender,education and loan id
X=X[selected_features]
X


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=0)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test)*100,"%")

**Again it can be seen that accuracy did not increased using feature selection**

So in totality, the accuracy we are getting is above 70 percent and our model is working pretty okay! But it can be concluded that data preprocessing steps are making significant changes positive or negative in the prediction.! It is not necessary that all the preprocessing steps increase the accuracy. We need to be carefull while thinking on what steps should be done and which ones should not be done.